
<a href="http://landlab.github.io"><img style="float: left; width: 300px;" src="https://landlab.csdms.io/_static/landlab_logo.png"></a>

# 2D Surface Water Flow component: Circular Dam Break

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>



## Overview

This notebook demonstrates the usage of the `RiverFlowDynamics` Landlab component to simulate a circular dam break. The component runs a semi-implicit, semi-Lagrangian finite-volume approximation to the depth-averaged 2D shallow-water equations of Casulli and Cheng (1992) and related work.

### Theory

The depth-averaged 2D shallow-water equations are the simplification of the Navier-Stokes equations, which correspond to the balance of momentum and mass in the fluid. It is possible to simplify these equations by assuming a well-mixed water column and a small water depth to width ratio, where a vertical integration results in depth-averaged equations. These require boundary conditions at the top and bottom of the water column, which are provided by the wind stress and the Manning-Chezy formula, respectively:

$$
\frac{\partial U}{\partial t}
+ U\frac{\partial U}{\partial x} + V\frac{\partial U}{\partial y}
= 
- g\frac{\partial \eta}{\partial x}
+ \epsilon\left(\frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2}\right)
+ \frac{\gamma_T(U_a - U)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}U + \mathbf{f}V
$$

$$
\frac{\partial V}{\partial t}
+ U\frac{\partial V}{\partial x} + V\frac{\partial V}{\partial y}
= 
- g\frac{\partial \eta}{\partial y}
+ \epsilon\left(\frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}\right)
+ \frac{\gamma_T(V_a - V)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}V + \mathbf{f}U
$$

$$
\frac{\partial \eta}{\partial t}
+ \frac{\partial (HU)}{\partial x} + \frac{\partial (HV)}{\partial y}
= 0
$$

where $U$ is the water velocity in the $x$-direction, $V$ is the water velocity in the $y$-direction, $H$ is the water depth, $\eta$ is the water surface elevation, $Cz$ is the Chezy friction coefficient, and $t$ is time. For the constants $g$ is the gravity acceleration, $\epsilon$ is the horizontal eddy viscosity, $\mathbf{f}$ is the Coriolis parameter, $\gamma_T$ is the wind stress coefficient, and $U_a$ and $V_a$ are the prescribed wind velocities.

### The component

Import the required libraries:

In [ ]:
import numpy as np
from IPython.display import clear_output
from landlab import RasterModelGrid
from landlab.components import RiverFlowDynamics
import matplotlib.pyplot as plt
from landlab.plot.imshow import imshow_grid
import os
from pathlib import Path

# Import plotting function
from river_flow_dynamics_tutorial4_plot import *


## Information about the component

Using the class name as argument for the `help` function returns descriptions of the various methods and parameters.

In [ ]:
help(RiverFlowDynamics)

## Example: Circular dam break

This example describes the instantaneous collapse of an idealized circular dam. The initial condition is comprised of two regions of stationary water bodies, which are separated by a thin-walled cylinder of diameter 22 m located in the center of a 50x50 m² horizontal and frictionless channel. The domain is discretized by 10,000 rectangular elements.

### Define simulation parameters

We specify basic parameters such as Manning's roughness coefficient, time step duration, number of time steps, and the domain dimensions.

In [ ]:
n = 0.012  # Manning's roughness coefficient, [s/m^(1/3)]
n_timesteps = 71  # Number of timesteps
dt = 0.01  # Timestep duration, [s]
nrows = 101  # Number of node rows
ncols = 101  # Number of node cols
dx = 0.5  # Node spacing in the x-direction, [m]
dy = 0.5  # Node spacing in the y-direction, [m]
theta = 0.5  # Implicit solution parameter

### Create the grid

We create a grid with the dimensions specified above.

In [ ]:
grid = RasterModelGrid((nrows, ncols), xy_spacing=(dx, dy))

### Set up topography

Create the elevation field and define the topography to represent a flat surface at the bottom.

In [ ]:
te = grid.add_zeros("topographic__elevation", at="node")

### Set initial water depth

The water depth inside and outside the wall are 11 and 1 m, respectively. We need to define this circular region carefully to avoid issues with square root calculations.

In [ ]:
# Initial water depth setup - Circular dam break
center_x = 25.0  # Grid center x
center_y = 25.0  # Grid center y
radius = 10.5    # Initial circle radius

# Calculate distance and create circle
distance_from_center = np.sqrt((grid.x_of_node - center_x)**2 + (grid.y_of_node - center_y)**2)
inside_circle = distance_from_center < radius

# Set water depth
h = grid.add_ones("surface_water__depth", at="node")
h[inside_circle] = 10.0

# Add transition front
transition_width = 0.5  # meters
transition_zone = (distance_from_center > radius) & (distance_from_center <= radius + transition_width)
transition_factor = 1 - (distance_from_center[transition_zone] - radius) / transition_width
h[transition_zone] = 1.0 + 9.0 * transition_factor

# Initialize velocity and water surface elevation
vel = grid.add_zeros("surface_water__velocity", at="link")
wse = grid.add_zeros("surface_water__elevation", at="node")
wse += h + te

### Visualize initial conditions

Let's check the initial water depth and water surface elevation before running the simulation.

In [ ]:
# Plotting water depth
plt.figure(figsize=(10, 8))
im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
plt.title("Initial Water Depth - (m)")
plt.show()

In [ ]:
# Plotting water surface elevation
plt.figure(figsize=(10, 8))
im = imshow_grid(grid, "surface_water__elevation", cmap="terrain")
plt.title("Initial Water Surface Elevation - (m)")
plt.show()

### Initialize the RiverFlowDynamics component

We initialize our component by passing the grid and simulation parameters.

In [ ]:
rfd = RiverFlowDynamics(grid, dt=dt, mannings_n=n, theta=theta)

### Run the simulation

Now we run the model to simulate the collapse of the circular dam and visualize the water spreading. An animation of the Dam Break will be generated. Animation options are set at the beginning of the cell.

In [ ]:
display_frequency = 1  # Show plots every N timesteps
output_dir = "simulation_frames" # Create output directory for images

Path(output_dir).mkdir(exist_ok=True)
# Main simulation loop
for timestep in range(n_timesteps):
    print(f"Running timestep {timestep+1}/{n_timesteps}")
    
    # Run one simulation step
    rfd.run_one_step()
    
    # Visualization and saving
    if timestep % display_frequency == 0:
        clear_output(wait=True)       
      
        # Create combined 3D and cross-section plot
        fig, (ax1, ax2, cbar_ax) = create_combined_plot(grid, timestep+1)
        
        # Save the figure as PNG with high quality
        filename = os.path.join(output_dir, f"frame_{timestep+1:03d}.png")
        fig.savefig(filename, dpi=150, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
        
        # Close the figure to free memory
        plt.close(fig)


### Visualize animation

Let's examine the water depth and water surface elevation evolution in time

In [ ]:
# Example 1: Create a 3-second animation
create_gif_animation(
    input_folder="simulation_frames",
    output_filename="dam_break.gif",
    total_duration_seconds=3
)
# Display it in the notebook
show_gif_in_notebook("dam_break.gif")

## Interpretation of Results

We can see how the initially circular dam with a flat top is now expanding across the domain. The results at the end of the simulation show that the water depth is propagating radially outward, creating complex flow patterns as it moves through the domain. 

Analytically, the water depth distribution should be symmetrical, but given our configuration to speed up the simulation, a relatively coarse grid was used. The observed asymmetries and small oscillations in the water depth are numerical artifacts resulting from this discretization. Using a finer grid resolution and smaller time steps would result in a more well-defined and symmetrical water depth distribution.

The cross-section plots reveal important hydraulic features:

1. The initial abrupt step in water depth (between 1m and 11m) has evolved into a smoother profile
2. Water is propagating outward as a wave front, with the highest elevation still near the center
3. Small oscillations in the water depth profile are characteristic of the second-order numerical scheme implemented in RiverFlowDynamics, which captures wave propagation with higher accuracy but can introduce these artifacts at sharp transitions

This simulation demonstrates the capability of the RiverFlowDynamics component to capture complex hydraulic phenomena such as dam breaks, flood waves, and their propagation over time.

## Physics of the dam break problem

This simulation demonstrates the classical dam break problem in a circular configuration. Initially, we have a higher water level inside a circular boundary (the "dam"), with a lower water level outside. When the simulation begins, the dam is instantly removed, allowing water to flow outward in all directions due to the pressure gradient.

Key phenomena observed in this simulation:

1. **Wave propagation**: As the dam breaks, a wave front propagates outward from the center.
2. **Momentum conservation**: The water carries momentum as it flows, creating complex flow patterns.
3. **Mass conservation**: The total volume of water in the system remains constant.
4. **Friction effects**: Manning's coefficient accounts for the frictional losses as water flows over the surface.

This type of simulation is valuable for understanding flood propagation, hydraulic jumps, and other hydrodynamic phenomena relevant to civil engineering and natural hazard assessment.

## Conclusion

In this tutorial, we've demonstrated how to use the `RiverFlowDynamics` component in Landlab to simulate a circular dam break. The component provides a robust solution to the shallow water equations, allowing for accurate modeling of complex hydraulic phenomena.

Some potential extensions to this example include:
- Adding topographic variations to the domain
- Changing the dam geometry to study different break patterns
- Adjusting Manning's coefficient to simulate different surface roughness conditions
- Adding inflow/outflow boundary conditions to simulate continuous flow

-- --
### And that's it! 

Nice work completing this tutorial. You now know how to use the `RiverFlowDynamics` Landlab component to run your own simulations :)

-- --


### Click here for more <a href="https://landlab.csdms.io/tutorials/">Landlab tutorials</a>